**Midterm**
First attemp to create homemade scraper

In [36]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

# NHL

## Overview

**1) Manual practice**

**2) Creating classes**

**3) Using developed classes**



In [5]:
requests.get("http://nhl.cz/sezona/tymy")

<Response [200]>

# 1) Manual practice

In [16]:
def getSoup(link):
    sleep(0.1) #to be kind to the website
    r = requests.get(link)
    r.encoding = 'UTF-8'
    return BeautifulSoup(r.text,'html')

In [19]:
# Getting club names from original website
def getClubName(link, cond):
    nhl = getSoup(link)
    teamContainer= nhl.findAll('div', {cond})
    return [team.find('h2').text for team in teamContainer]

In [20]:
# Application of getClubName()
link = "http://nhl.cz/sezona/tymy"
cond = 'box-team__container'

clubNames = getClubName(link, cond)
clubNames

['Anaheim Ducks',
 'Arizona Coyotes',
 'Boston Bruins',
 'Buffalo Sabres',
 'Calgary Flames',
 'Carolina Hurricanes',
 'Colorado Avalanche',
 'Columbus Blue Jackets',
 'Dallas Stars',
 'Detroit Red Wings',
 'Edmonton Oilers',
 'Florida Panthers',
 'Chicago Blackhawks',
 'Los Angeles Kings',
 'Minnesota Wild',
 'Montreal Canadiens',
 'Nashville Predators',
 'New Jersey Devils',
 'New York Islanders',
 'New York Rangers',
 'Ottawa Senators',
 'Philadelphia Flyers',
 'Pittsburgh Penguins',
 'San Jose Sharks',
 'St. Louis Blues',
 'Tampa Bay Lightning',
 'Toronto Maple Leafs',
 'Vancouver Canucks',
 'Vegas Golden Knights',
 'Washington Capitals',
 'Winnipeg Jets']

In [21]:
def getTeamLinks(link):
    nhl = getSoup(link)
    teamContainer= nhl.findAll('div', {cond})
    return ['http://nhl.cz' + team.find('a')['href'] for team in teamContainer]

links = getTeamLinks("http://nhl.cz/sezona/tymy")
links

['http://nhl.cz/klub/2004',
 'http://nhl.cz/klub/2005',
 'http://nhl.cz/klub/1987',
 'http://nhl.cz/klub/1989',
 'http://nhl.cz/klub/2006',
 'http://nhl.cz/klub/1990',
 'http://nhl.cz/klub/2008',
 'http://nhl.cz/klub/1991',
 'http://nhl.cz/klub/2009',
 'http://nhl.cz/klub/1992',
 'http://nhl.cz/klub/2010',
 'http://nhl.cz/klub/1993',
 'http://nhl.cz/klub/2007',
 'http://nhl.cz/klub/2011',
 'http://nhl.cz/klub/2012',
 'http://nhl.cz/klub/1994',
 'http://nhl.cz/klub/2013',
 'http://nhl.cz/klub/1995',
 'http://nhl.cz/klub/1996',
 'http://nhl.cz/klub/1997',
 'http://nhl.cz/klub/1998',
 'http://nhl.cz/klub/1999',
 'http://nhl.cz/klub/2000',
 'http://nhl.cz/klub/2014',
 'http://nhl.cz/klub/2015',
 'http://nhl.cz/klub/2001',
 'http://nhl.cz/klub/2002',
 'http://nhl.cz/klub/2016',
 'http://nhl.cz/klub/2655',
 'http://nhl.cz/klub/2003',
 'http://nhl.cz/klub/2017']

In [22]:
# Different method for obtaining club name - straight from the club page
def getClubNameAgain(link):
    soup = getSoup(link)
    return soup.find('h1').text
clubNamesAgain = [getClubNameAgain(link) for link in links[:3]]
clubNamesAgain

['Anaheim Ducks',
 'Arizona Coyotes',
 'Boston Bruins',
 'Buffalo Sabres',
 'Calgary Flames',
 'Carolina Hurricanes',
 'Colorado Avalanche',
 'Columbus Blue Jackets',
 'Dallas Stars',
 'Detroit Red Wings',
 'Edmonton Oilers',
 'Florida Panthers',
 'Chicago Blackhawks',
 'Los Angeles Kings',
 'Minnesota Wild',
 'Montreal Canadiens',
 'Nashville Predators',
 'New Jersey Devils',
 'New York Islanders',
 'New York Rangers',
 'Ottawa Senators',
 'Philadelphia Flyers',
 'Pittsburgh Penguins',
 'San Jose Sharks',
 'St. Louis Blues',
 'Tampa Bay Lightning',
 'Toronto Maple Leafs',
 'Vancouver Canucks',
 'Vegas Golden Knights',
 'Washington Capitals',
 'Winnipeg Jets']

In [46]:
# Get info about the best club players
def getBestPlayers(link):
    soup = getSoup(link)
    statsContainer= soup.findAll('div', {"person__description"})
    return [player.find('a').text for player in statsContainer]


In [67]:
[getBestPlayers(link) for link in links[:3]]

[['Adam Henrique',
  'Adam Henrique',
  'Ryan Getzlaf',
  'Michael Del Zotto',
  'John Gibson'],
 ['Taylor Hall',
  'Conor Garland',
  'Taylor Hall',
  'Alex Goligoski',
  'Darcy Kuemper'],
 ['David Pastrňák',
  'David Pastrňák',
  'Brad Marchand',
  'Zdeno Chára',
  'Tuukka Rask']]

# 2) Creating classes

## IES example in collapsed cells:

In [ ]:
class IES_Person(IES_Web):
    '''
    Class representing a personal website of IES researchers, Ph.D students etc.
    
    Inherits from the IES_Web class, so that it access its XML parsing methods
    '''
    def __init__(self,link,category):
        '''
        Constructor for IES_Person calls parents IES_Web constructor first,
        where self.link, self.request and self.soup are created
        
        Then unique id, name and characteristics are generated as IES_Person attributes
        '''
        
        # calling a parents IES_Web constructor
        super().__init__(link)
        
        # read Persons ID from link
        self.id = self.linkToID()
        
        # Is he internal, external, administrative or Ph.D. candidate?
        self.category = category
        
        # read persons name from <h2>
        self.name = self.parseH2()
        
        # get the rest of informations
        self.characteristics = self.parseCharacteristics()
    
    def linkToID(self):
        '''
        Person IDs are derived from the last part of the his personal website
        '''
        llink = self.link.split('/')
        return llink[len(llink)-1]
    
    def parseCharacteristics(self):
        '''
        returns pandas series with IES_Person characteristics
        
        See documentation of individual methods methods for details
        and example page: http://ies.fsv.cuni.cz/cs/staff/barunik
        '''
        
        # Generate first pandas series - see ?self.pdSiblingsOfStrong
        nextSiblings = ['Position:','Field of interest:','Membership:','Office:','Email:','Phone:','Available:']       
        ser_nextSiblings = self.pdSiblingsOfStrong(nextSiblings)
        
        # Generate second pandas series - see ?self.pdSiblingsOfStrongParents
        parentsNextSiblings = ['Organisation Memberships','Education','Job history','Extra activities','Bachelor theses','Master theses']
        ser_parentsNextSiblings = self.pdSiblingsOfStrongParents(parentsNextSiblings)
        
        # merge first and second together
        chars = pd.concat([ser_nextSiblings,ser_parentsNextSiblings])
        
        # add object details
        chars.loc['researcher'] = self.name
        chars.loc['id'] = self.id
        chars.loc['category'] = self.category
        
        # parse bachelor and master thesis counts (both all and marked)
        d = {'bachelor':'Supervised Bachelor theses','master':'Supervised Master Theses'}
        for key in d:
            total, awarded = self.numbersNextToStrongBelowH3(d[key])
            chars.loc[key + '_all'] = total
            chars.loc[key + '_awarded'] = awarded
        return chars

In [77]:
def downloadPeople(self,pause=0.5):
        '''
        Download all links stored in self.link['people'] and store it in self.people
        pause -- how long to pause between requests? (in seconds)
        tqdm -- the progress bar showing a progress of iterator
        '''
        if self.allowLog:
            count = sum([len(self.links['people'][key])for key in self.links['people']])
            print('Downloading all {} persons ...'.format(count))
        for key in self.links['people']:
            for link in tqdm(self.links['people'][key],desc=key):
                person = IES_Person(link,key)
                self.people.append(person)
                time.sleep(pause)
        if self.allowLog:
            print('Succesfully downloaded {} persons'.format(len(self.people)))

In [38]:
class NHL_Club:
    '''
    Class representing a NHL team webpage.
    '''
    def __init__(self, link):
        self.link = link
        r = requests.get(link)
        r.encoding='UTF-8'
        self.soup = BeautifulSoup(r.text,'html')
        
        # read club ID from link
        self.id = self.linkToID()
                       
        # read club name from <h1>
        self.clubName = self.getClubNameAgain()
        
        # get names of best players
        self.bestPlayers = self.getBestPlayers()
    
    def linkToID(self):
        '''
        Team IDs are derived from the last part of the his personal website
        '''
        llink = self.link.split('/')
        return llink[len(llink)-1]
    
    def getClubNameAgain(self):
        '''
        Organization's name is derived from <h1> headline on club webpage.
        '''
        return self.soup.find('h1').text
    
    def getBestPlayers(self):
        statsContainer= self.soup.findAll('div', {"person__description"})
        return [player.find('a').text for player in statsContainer]

        

In [24]:
link = "http://nhl.cz/sezona/tymy"

In [52]:
class NHL_Downloader:
    '''
    Download manager class for NHL web
    
    It contains methods for collection of links, downloading itself and storing results
    '''
    
    def __init__(self, link):
        '''
        creates NHL_Downloader object with self.links, that store links to be downloaded
        and self.clubs store individual IES_Web-like objects
        '''
        
        self.links = getTeamLinks(link)
        self.clubs = []
        
    def getNHLSoup(self, link):
        '''
        Getting soup of NHL domain.
        '''
        sleep(0.1) #to be kind to the website
        r = requests.get(link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text,'html')

    def getTeamLinks(self, link):
        '''
        obtain a link for each NHL club website
        '''        
        nhl = getNHLSoup(link)
        teamContainer= nhl.findAll('div', {'box-team__container'})
        return ['http://nhl.cz' + team.find('a')['href'] for team in teamContainer]
    
        
    
    def downloadCLubs(self,pause=0.5):
        '''
        Download all links stored in self.link and store it in self.clubs
        pause -- how long to pause between requests? (in seconds)
        tqdm -- the progress bar showing a progress of iterator
        '''

        for link in self.links:
            club = NHL_Club(link)
            self.clubs.append(club)
            time.sleep(pause)

    def saveDF(self):
        df = pd.DataFrame([x.bestPlayers for x in self.clubs])    
        df.columns = ['Points', 'Goals', 'Assists', '+/-', '%SV']
        df.index = NHL_Club.getClubNameAgain
        self.df = df
        


# 3) Using developed classes

In [53]:
dl = NHL_Downloader(link)

In [56]:
dl.downloadCLubs()

In [78]:
dl.saveDF()
dl.df.columns = ['Points', 'Goals', 'Assists', '+/-', 'SV%']
dl.df.index = clubNamesAgain

In [79]:
dl.df

,Points,Goals,Assists,+/-,SV%
Anaheim Ducks,Adam Henrique,Adam Henrique,Ryan Getzlaf,Michael Del Zotto,John Gibson
Arizona Coyotes,Taylor Hall,Conor Garland,Taylor Hall,Alex Goligoski,Darcy Kuemper
Boston Bruins,David Pastrňák,David Pastrňák,Brad Marchand,Zdeno Chára,Tuukka Rask
Buffalo Sabres,Jack Eichel,Jack Eichel,Jack Eichel,Brandon Montour,Linus Ullmark
Calgary Flames,Matthew Tkachuk,Elias Lindholm,Johnny Gaudreau,Derek Ryan,Cam Talbot
Carolina Hurricanes,Sebastian Aho,Sebastian Aho,Teuvo Teräväinen,Dougie Hamilton,James Reimer
Colorado Avalanche,Nathan MacKinnon,Nathan MacKinnon,Nathan MacKinnon,Ryan Graves,Pavel Francouz
Columbus Blue Jackets,Pierre-Luc Dubois,Oliver Bjorkstrand,Pierre-Luc Dubois,Seth Jones,Elvis Merzlikins
Dallas Stars,Tyler Seguin,Denis Gurjanov,Tyler Seguin,Miro Heiskanen,Anton Chudobin
Detroit Red Wings,Dylan Larkin,Tyler Bertuzzi,Dylan Larkin,Brian Lashoff,Jonathan Bernier
